In [2]:
import json
import yaml
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy as sq
from sqlalchemy.ext.declarative import declarative_base
import os
from loguru import logger

In [3]:
def make_db_connection_psycopg2(database: str, autocommit: bool = False):

    #assert database in list(conn_dict.keys()), "server couldn't be recognized in the config file"

    
    database="blog"
    host="localhost"
    port="5432"
    user="postgres"
    password="matin9862"
    
    conn = psycopg2.connect(user=user, password=password, host=host, dbname=database)
    cursor = conn.cursor()

    if autocommit:
        conn.autocommit = True

    return conn, cursor


In [4]:
conn, curs = make_db_connection_psycopg2(database="blog")

In [20]:
conn

<connection object at 0x7f815c7e8680; dsn: 'user=postgres password=xxx dbname=blog host=localhost', closed: 0>

In [19]:
conn, cursor = make_db_connection_psycopg2("blog")
# query = """
# SELECT * FROM information_schema.tables WHERE table_schema = 'public';
# "

query = """
SELECT * FROM home_blog;

"""

cursor.execute(query)
rows = cursor.fetchall()

for i in rows:
    print(i)

(1, 'rings', 'rings of lords', 'c.zhzkhdh.vx.dkbvxvxv', datetime.datetime(2024, 1, 11, 19, 1, 3, 266020, tzinfo=datetime.timezone.utc), datetime.time(19, 1), 1)
